In [1]:
import gradio as gr
import torch
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
import imageio
import numpy as np
from pathlib import Path

Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.
Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.


In [2]:
@torch.no_grad()
def load_model():
    model_id = "cerspense/zeroscope_v2_576w"

    pipe = DiffusionPipeline.from_pretrained(
        model_id,
        torch_dtype=torch.float16
    )
    pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
    pipe.enable_model_cpu_offload()

    return pipe

pipe = load_model()
print('model loaded')

model_index.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

text_encoder/pytorch_model.bin:   0%|          | 0.00/681M [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

scheduler_config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

...

vae/diffusion_pytorch_model.bin:   0%|          | 0.00/167M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

An error occurred while trying to fetch /root/.cache/huggingface/hub/models--cerspense--zeroscope_v2_576w/snapshots/6963642a64dbefa93663d1ecebb4ceda2d9ecb28/vae: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--cerspense--zeroscope_v2_576w/snapshots/6963642a64dbefa93663d1ecebb4ceda2d9ecb28/vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
`torch_dtype` is deprecated! Use `dtype` instead!
An error occurred while trying to fetch /root/.cache/huggingface/hub/models--cerspense--zeroscope_v2_576w/snapshots/6963642a64dbefa93663d1ecebb4ceda2d9ecb28/unet: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--cerspense--zeroscope_v2_576w/snapshots/6963642a64dbefa93663d1ecebb4ceda2d9ecb28/unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
The TextToVideoSDPipeline has been deprecated and wi

model loaded


In [3]:
def generate_video(prompt, negative_prompt="", num_frames=16, num_inference_steps=25):
    try:
        video_frames = pipe(
            prompt=prompt,
            negative_prompt=negative_prompt,
            num_frames=num_frames,
            num_inference_steps=num_inference_steps,
            height=256,
            width=256,
            guidance_scale=9.0
        ).frames[0]
        output_path = "output_video.mp4"
        video_frames = [(frame * 255).astype(np.uint8) for frame in video_frames]
        imageio.mimsave(output_path, video_frames, fps=8)

        return output_path

    except Exception as e:
        return f"Error: {str(e)}"

In [ ]:
with gr.Blocks(title="Text to Video Generator") as demo:
    gr.Markdown("# Text to Video Generator")
    gr.Markdown("Generate short videos from text descriptions using AI.")

    with gr.Row():
        with gr.Column():
            prompt_input = gr.Textbox(
                label="Prompt",
                placeholder="A cat walking on the beach at sunset...",
                lines=3
            )
            negative_prompt_input = gr.Textbox(
                label="Negative Prompt (Optional)",
                placeholder="blurry, low quality...",
                lines=2
            )

            with gr.Row():
                num_frames = gr.Slider(
                    minimum=8,
                    maximum=24,
                    value=16,
                    step=8,
                    label="Number of Frames"
                )
                num_steps = gr.Slider(
                    minimum=10,
                    maximum=50,
                    value=25,
                    step=5,
                    label="Inference Steps"
                )

            generate_btn = gr.Button("Generate Video", variant="primary")

        with gr.Column():
            video_output = gr.Video(label="Generated Video")

    gr.Examples(
        examples=[
            ["A golden retriever playing in the snow"],
            ["Astronaut riding a horse in space"],
            ["Fireworks exploding over a city at night"],
            ["A butterfly landing on a flower in slow motion"]
        ],
        inputs=prompt_input
    )

    generate_btn.click(
        fn=generate_video,
        inputs=[prompt_input, negative_prompt_input, num_frames, num_steps],
        outputs=video_output
    )
demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://9bddb125c4ca9afe97.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


  0%|          | 0/50 [00:00<?, ?it/s]